In [1]:
import sys
print(sys.version)


3.6.8 |Anaconda, Inc.| (default, Dec 29 2018, 19:04:46) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]


In [2]:
import numpy as np
from dipy.io.streamline import load_trk
from dipy.viz import window, actor
import math
from nibabel import trackvis as tv
from dipy.tracking.streamline import Streamlines
from dipy.segment.clustering import QuickBundles
from dipy.io.pickles import save_pickle
from dipy.data import get_data
from dipy.segment.metric import mdf
from dipy.segment.metric import AveragePointwiseEuclideanMetric
import matplotlib.pyplot as plt
import pickle
from dipy.io.streamline import load_trk, save_trk
import nibabel

In [3]:
def show_streamlines(streamlines): #function to visualize streamlines
    
    ren = window.Renderer()
    ren.add(actor.line(streamlines))
    window.show(ren)

In [4]:
def show_multiple_streamlines(streamline1,streamline2): #function to visualize streamlines
    
    ren = window.Renderer()
    ren.add(actor.line(streamline1))
    ren.add(actor.line(streamline2,colors=(0,1,1)))
    window.show(ren)

 load brain (tractogram)

In [5]:
# brain, b_hr = load_trk("brain.trk")
brain=[]

load Arcuate Fasciculus left fiber bundle (AF_L)

In [6]:
# bundle, bn_hr = load_trk("AF_L.trk")
# brain.append(bundle)

In [7]:
import os
os.getcwd()

'/Users/rp7/Documents/FiberBundles'

In [8]:
# load data
file = nibabel.streamlines.load("data/SLF_I_right.tck")
save_trk("SLF_I_right.trk",file.streamlines, np.eye(4))
bundle, bn_hr = load_trk("SLF_I_right.trk")
brain.append(bundle[:500])

FileNotFoundError: [Errno 2] No such file or directory: 'data/SLF_I_right.tck'

In [9]:
file = nibabel.streamlines.load("OR_left.tck")
save_trk("OR_left.trk",file.streamlines, np.eye(4))
bundle, bn_hr = load_trk("OR_left.trk")
brain.append(bundle[:500])

FileNotFoundError: [Errno 2] No such file or directory: 'OR_left.tck'

In [10]:
file = nibabel.streamlines.load("ICP_left.tck")
save_trk("ICP_left.trk",file.streamlines, np.eye(4))
bundle, bn_hr = load_trk("ICP_left.trk")
brain.append(bundle[:500])

FileNotFoundError: [Errno 2] No such file or directory: 'ICP_left.tck'

In [6]:
# file = nibabel.streamlines.load("data/FX_right.tck")
# save_trk("FX_right.trk",file.streamlines, np.eye(4))
bundle, bn_hr = load_trk("data/FX_right.trk")
brain.append(bundle[:500])

In [7]:
# cst, bn_hr = load_trk("CST_recognized.trk")

let's visualize them

In [9]:
# first brain

show_streamlines(bundle)
# show_multiple_streamlines(brain[1953:1954], brain[1950+81:1950+82])

In [123]:
# now bundle
brain = np.array(brain).flatten()


In [124]:
show_streamlines(brain)

In [93]:
(bundle[0].shape)
len(bundle[0])

331

In [94]:
# get labels

def getLabels(brain, bundle):
    labels = np.zeros(len(brain))
    npbrain = np.array(brain)
    for i in range(len(bundle)):
        for j in range(len(brain)):
            if((npbrain[j].shape[0]==bundle[i].shape[0])):
    #         print(i,j)
                if np.allclose(npbrain[j], bundle[i]):
                    labels[j] = 1
                    break
        if(i%100==0):
            print(i,end=" ")
    return labels

In [125]:
labels = np.zeros(len(brain))
l = getLabels(brain, brain[:50])
labels[l==l] = 0
l = getLabels(brain, brain[50:100])
labels[l==l] = 1
l = getLabels(brain, brain[100:150])
labels[l==l] = 2
l = getLabels(brain, brain[150:200])
labels[l==l] = 3

In [14]:
#Find Frechet distance
# From : https://gist.github.com/MaxBareiss/ba2f9441d9455b56fbc9

# Euclidean distance.
def euc_dist(pt1,pt2):
    return math.sqrt((pt2[0]-pt1[0])*(pt2[0]-pt1[0])+(pt2[1]-pt1[1])*(pt2[1]-pt1[1]))

def _c(ca,i,j,P,Q):
    if ca[i,j] > -1:
        return ca[i,j]
    elif i == 0 and j == 0:
        ca[i,j] = euc_dist(P[0],Q[0])
    elif i > 0 and j == 0:
        ca[i,j] = max(_c(ca,i-1,0,P,Q),euc_dist(P[i],Q[0]))
    elif i == 0 and j > 0:
        ca[i,j] = max(_c(ca,0,j-1,P,Q),euc_dist(P[0],Q[j]))
    elif i > 0 and j > 0:
        ca[i,j] = max(min(_c(ca,i-1,j,P,Q),_c(ca,i-1,j-1,P,Q),_c(ca,i,j-1,P,Q)),euc_dist(P[i],Q[j]))
    else:
        ca[i,j] = float("inf")
    return ca[i,j]

""" Computes the discrete frechet distance between two polygonal lines
Algorithm: http://www.kr.tuwien.ac.at/staff/eiter/et-archive/cdtr9464.pdf
P and Q are arrays of 2-element arrays (points)
"""
def frechetDist(P,Q):
    ca = np.ones((len(P),len(Q)))
    ca = np.multiply(ca,-1)
    return _c(ca,len(P)-1,len(Q)-1,P,Q)

In [15]:
# Fast Dynamic time warping
# From: https://github.com/slaypni/fastdtw

import numpy as np
from fastdtw import fastdtw

In [16]:
from dipy.align.streamlinear import BundleMinDistanceMetric
from dipy.align.streamlinear import set_number_of_points
# BundleMinDistanceMetric().distance(cBrain3D[0],cBrain3D[1])

def bmd(a,b):
    static = set_number_of_points(a, len(a))
    moving = set_number_of_points(b, len(b))
    BMD = BundleMinDistanceMetric()
    BMD.setup(static, moving)
    x0 = np.array([0, 0, 0, 0, 0, 0, 1., 1., 1, 0, 0, 0])  # affine
    bmd_value = BMD.distance(x0.tolist())
    return bmd_value

In [17]:
# bmd(brain3D[0],brain3D[7])
# brain3D[1].shape

In [18]:
# Get distance between 2 curves
from scipy.spatial.distance import euclidean

def distanceBetweenCurves(c1, c2, algo='fastdtw'):
    if algo=='fastdtw':
        distance, path = fastdtw(c1,c2, radius = 1, dist = euclidean) # check out the 'radius' parameter to tune the distance
        return distance
    elif algo=='frechet':
        return frechetDist(c1,c2)
    elif algo=='bmd':
        return bmd(c1,c2)
    elif algo=='mdf':
        return mdf(c1,c2)

In [19]:
# project streamline on 2D plane using PCA
from sklearn.decomposition import PCA
def project2D(streamline):
    pca = PCA(n_components=2)
    newBundle = pca.fit_transform(streamline)
#     newStreamline = Streamlines([(np.hstack((newBundle,np.zeros((newBundle.shape[0])).reshape((-1,1)))))])
    newStreamline = Streamlines([newBundle])
    return newStreamline, pca.components_, pca.singular_values_

In [20]:
# print(pca.explained_variance_ratio_) 
# print(pca.singular_values_)

In [21]:
# convert to 1D data using the angle representation
def convertToAngles(streamline2D):
    streamline2D = np.array(streamline2D)
#     print((streamline2D))
    angles = np.zeros(streamline2D.shape[0])
    angles[0] = 1
#     print(streamline2D.shape)
    # handle the direction of start
#     print(streamline2D.shape)
    for point in range(1,streamline2D.shape[0]):
#         print(point)
        diff = streamline2D[point] - streamline2D[point-1]
#         print(streamline2D[point], streamline2D[point-1])
        if(diff[0]==0):
            angles[point] = angles[point-1]
        else:
            angles[point] = math.degrees(math.atan(diff[1]/diff[0]))
    return angles


In [22]:
# center the data
def centerData(data):
    for i in range(len(data)):
        mean = np.mean(data[i],axis=0)
        data[i] = data[i]-mean
    return data

In [23]:
# get lower dimensional data(1D, 2D)
def getDataLD(bundle):
    data2D = []
    angularData = []
    eigenValues = []
    eigenVecs = []
    for index in range(len(bundle)):
        newStreamline, vecs, singVal = project2D(np.array(bundle[index]))
        data2D.append(newStreamline[0][:,:2])
        eigenValues.append(singVal)
        eigenVecs.append(vecs)
        # append these values to store them 
        streamlineAsAngle = convertToAngles(newStreamline[0])
#         print(streamlineAsAngle)
        angularData.append(streamlineAsAngle)
    
    return angularData, data2D, eigenVecs, eigenValues

In [24]:
cBrain = centerData(np.array(brain))
brain1D, brain2D, eigenVecs, eigenValues = getDataLD(cBrain)
brain3D = np.array(cBrain).tolist()
# show_multiple_streamlines(newStreamline,bundle[0:1])

In [25]:
# From https://www.geeksforgeeks.org/program-to-find-equation-of-a-plane-passing-through-3-points/
# get equation of plane from 3 points
def getPlaneEquation(p1, p2, p3):    
    a1 = p2[0] - p1[0] 
    b1 = p2[1] - p1[1] 
    c1 = p2[2] - p1[2] 
    a2 = p3[0] - p1[0] 
    b2 = p3[1] - p1[1] 
    c2 = p3[2] - p1[2]
    a = b1 * c2 - b2 * c1 
    b = a2 * c1 - a1 * c2 
    c = a1 * b2 - b1 * a2 
    d = (- a * p1[0] - b * p1[1] - c * p1[2]) 
    norm = np.sqrt(a**2+b**2+c**2)
    return a/norm,b/norm,c/norm,d/norm
#     print "equation of plane is ", 
#     print a, "x +", 
#     print b, "y +", 
#     print c, "z +", 
#     print d, "= 0."


In [26]:
# brain data in a particular plane but in 3D space
x2DBrain3D = [(np.dot(eigenVecs[i].T, stream.T)).T for i, stream in enumerate(brain2D)]
# (eigenVecs[0]).shape

In [112]:
len(x2DBrain3D)

200

In [32]:
show_multiple_streamlines(x2DBrain3D[65:66],brain3D[65:66])

In [34]:
getPlaneEquation(x2DBrain3D[4][5], x2DBrain3D[4][6], x2DBrain3D[4][2])

(0.9858762976317351,
 0.0679287596567005,
 0.15308040168305262,
 0.3605892652705367)

In [35]:
# find vector at an angle w.r.t. the given vector and rotated along the axis
def rodriguesRotation(axis, vector, angle):
    # axis should be a unit vector
    return vector*np.cos(angle) + (np.cross(axis, vector))*np.sin(angle) + axis*(np.dot(axis,vector))*(1-np.cos(angle))

In [36]:
rodriguesRotation(np.array([0,1,0]), np.array([1,0,0]), 2*np.pi)

array([1.0000000e+00, 0.0000000e+00, 2.4492936e-16])

In [37]:
atheta = 0
ztheta = 0
subSegments = 7 # using odd value to prevent redunduncy in the discrete planes
subAngles = np.pi/subSegments
noOfPlanes = subSegments*subSegments
discretePlanes = []
for a in range(subSegments):
    vec = np.array([np.cos(atheta+(a*subAngles)), np.sin(atheta+(a*subAngles)), 0])
    axis = np.array([np.cos((np.pi/2)+atheta+(a*subAngles)), np.sin((np.pi/2)+atheta+(a*subAngles)), 0])
    for z in range(subSegments):
        discretePlanes.append(rodriguesRotation(axis, vec, ztheta+(z*subAngles)))

In [38]:
discretePlanes = np.array(discretePlanes)

In [39]:
# (brain3D[0]).shape
# temp = Streamlines([(np.hstack((brain2D[3],np.zeros((brain2D[3].shape[0])).reshape((-1,1)))))])
# show_multiple_streamlines(temp,tempStream)
# print(len(brain1D),len(brain2D),len(brain3D))

In [126]:
# np.dot((eigenVecs[3][1]), (eigenVecs[3][2]))

In [127]:
distanceBetweenCurves(brain3D[3], brain3D[81], 'fastdtw')

27224.70919387607

In [129]:
# create covariance matrix

cov = np.zeros((len(brain3D), len(brain3D)))
for i in range(len(brain3D)):
    for j in range(len(brain3D)):
        cov[i][j] = distanceBetweenCurves(brain3D[i], brain3D[j], 'fastdtw')
    print(i, end=" ")

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

In [134]:
# import matplotlib.pyplot as plt
# %matplotlib qt
# plt.imshow(cov)

In [2]:
# get centroids for the bundles

%time
def getCentroids(streamlines, noOfCentroids, pointsInCentroid=400):
    streamlines = [set_number_of_points(streamline, nb_points=pointsInCentroid) for streamline in streamlines]
    thresh=1
    qb = QuickBundles(threshold=thresh, metric=AveragePointwiseEuclideanMetric())
    clusters = qb.cluster(streamlines)
    length = len(clusters.centroids)
    prev = 0
    flag=False
    while(length!=noOfCentroids):
        if(length>noOfCentroids):
            if flag:
                temp = thresh
                thresh= thresh + (thresh-prev)/2.0
                prev = temp
            else:
                prev=thresh
                thresh *= 2.0
        else:
            temp = thresh
            thresh = prev+(thresh-prev)/2.0
            flag = True
        qb = QuickBundles(threshold=thresh, metric=AveragePointwiseEuclideanMetric())
        clusters = qb.cluster(streamlines)
        length = len(clusters)
    return np.array(clusters.centroids)

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 7.87 µs


In [3]:
%time
def getAllCentroids(brainData, labels, noOfBundles, noOfCentroids=3):
    centroids = []
    for n in range(noOfBundles):
        print(n)
        if n==0:
            c = getCentroids(brainData[labels==n], noOfCentroids)
            centroids = c.reshape((1,)+c.shape)
        else:
            centroids = np.vstack((centroids, getCentroids(brainData[labels==n], noOfCentroids).reshape((1,)+c.shape)))
    return centroids

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 8.11 µs


In [55]:
(np.unique(labels))

array([0., 1., 2., 3.])

In [4]:
# shape = (no of bundles, no of centroids in each bundle, no of points in each centroid, dimensions of the points)
centroidMat = getAllCentroids(np.array(brain3D),labels, len(np.unique(labels)))


NameError: name 'np' is not defined

In [126]:
cent

In [58]:
# get distance of a streamline from all the centroids
def getDistToCentroids(streamline):
    global centroidMat
    distMat = np.zeros((centroidMat.shape[0], centroidMat.shape[1]))
    for i in range(distMat.shape[0]):
        for j in range(distMat.shape[1]):
            distMat[i][j] = distanceBetweenCurves(centroidMat[i][j], streamline, 'fastdtw')
    return distMat

In [59]:
# get probability of getting a bundle given a streamline(shape)
def pB_S(streamline, metric='min'):
    distMat = getDistToCentroids(streamline)
    if metric=='min':
        distToBundles = np.min(distMat, axis=1)
    elif metric=='avg':
        distToBundles = np.mean(distMat, axis=1)
    
    probabilities = 1/distToBundles
    probabilities /= np.sum(probabilities)
    return probabilities

In [130]:
count=[]
predictions1 = []
predictions2 = []
predictions3 = []
predictions0 = []
for i in range(200):
    cls = ((pB_S(brain3D[i])))
    if np.argmax(cls)==0:
        predictions0.append(brain3D[i])
    if np.argmax(cls)==1:
        predictions1.append(brain3D[i])
    if np.argmax(cls)==2:
        predictions2.append(brain3D[i])
    if np.argmax(cls)==3:
        predictions3.append(brain3D[i])
    count.append(np.argmax(cls))
#     print(cls)

In [226]:
show_streamlines(predictions)

/Users/rp7/anaconda3/lib/python3.6/site-packages/vtk/util/numpy_support.py:137: FutureWarning: Conversion of the second argument of issubdtype from `complex` to `np.complexfloating` is deprecated. In future, it will be treated as `np.complex128 == np.dtype(complex).type`.
  assert not numpy.issubdtype(z.dtype, complex), \


In [73]:
count = np.array(count)
(np.sum(count[:50]==0) + np.sum(count[50:100]==1) + np.sum(count[100:150]==2) + np.sum(count[150:]==3))/200


0.485

In [74]:
# probability of a centroid given a particular streamline(shape)
# dimensions: (no of bundles, no of centroids)
def pC_S(streamline):
    dist = 1/getDistToCentroids(streamline)
    return dist/np.sum(dist)

In [76]:
# get dihedral angle between 2 planes
def dihedralAngle(a, b):
    temp =  np.arccos((np.dot(a,b))/(np.linalg.norm(a)*np.linalg.norm(b)))
    if temp>10000:
        print(a,b)
    return temp


In [77]:
# get the probability of getting the discrete set of planes given the centroids
# dimensions: (total number of centroids, number of discrete planes)
def pP_C():
    noOfPlanes = len(discretePlanes)
    probabilities = np.zeros((centroidMat.shape[0]*centroidMat.shape[1], noOfPlanes))
    for i in range(centroidMat.shape[0]):
        for j in range(centroidMat.shape[1]):
            cent, evecs, evalue = project2D(centerData(centroidMat[i][j]))
            PCAedCentroid = (np.dot(evecs.T, np.array(cent[0]).T)).T
            a,b,c,d = getPlaneEquation(PCAedCentroid[0],PCAedCentroid[1],PCAedCentroid[2])
            for k in range(noOfPlanes):
                probabilities[centroidMat.shape[1]*i+j][k] = dihedralAngle(np.array([a,b,c]), discretePlanes[k])
            
    probabilities = 1.0/probabilities
    probabilities /= np.sum(probabilities, axis=1).reshape((-1,1))
    return probabilities
    

In [78]:
dihedralAngle(np.array([-0.5775682004691819, -0.5773971386380066, -0.5770853646553805]), discretePlanes[4])

0.8077343472643393

In [79]:
pPC = pP_C()

In [80]:
centroidMat.shape

(4, 3, 400, 3)

In [81]:
# probability of getting a bundle given a plane
# dimensions: (number of discrete planes, no of bundles)
def pB_P(pPC, metric='avg'):
    noOfPlanes = len(discretePlanes)
    angles = np.zeros((centroidMat.shape[0]*centroidMat.shape[1], noOfPlanes))
#     centDist = np.sum(pPC, axis=1)
    for i in range(centroidMat.shape[0]):
        for j in range(centroidMat.shape[1]):
            cent, evecs, evalue = project2D(centerData(centroidMat[i][j]))
            PCAedCentroid = (np.dot(evecs.T, np.array(cent[0]).T)).T
            a,b,c,d = getPlaneEquation(PCAedCentroid[0],PCAedCentroid[1],PCAedCentroid[2])
            for k in range(noOfPlanes):
                angles[centroidMat.shape[1]*i+j][k] = dihedralAngle(np.array([a,b,c]), discretePlanes[k])
    probabilities = np.zeros((noOfPlanes, centroidMat.shape[0]))
    for i in range(probabilities.shape[1]):
        probabilities[:,i] = np.sum(angles[i*centroidMat.shape[1]:(i+1)*centroidMat.shape[1],:],axis=0)
#     print(np.sum(probabilities, axis=1).shape)
    probabilities = 1.0/probabilities
    probabilities /= np.sum(probabilities, axis=1).reshape((-1,1))
    
    return probabilities
    

In [82]:
# probability of Bundle given centroid and plane information
# dimensions: ((no of centroids*no of discrete planes), no of bundles)
def pB_PC(pBP, plane, centroid):
    return pBP[plane]*pPC[centroid][plane]

In [83]:
pBP = pB_P(pPC)

In [92]:
# Gibbs Sampling/MCMC
def predict(streamline):
    l = np.argmax(np.random.random(centroidMat.shape[0]))
    answers = []
    probc = pC_S(streamline).flatten()
    prevb = 0
    for i in range(5000):
        c = np.random.choice(np.arange(probc.shape[0]), p=probc)
        probp = pPC[c,:]
        p = np.random.choice(np.arange(probp.shape[0]), p=probp)
        prob = pB_PC(pBP, p, c)
        finalProb = prob*probc[c]*probp[p]
        finalProb/=np.sum(finalProb)
        l = (np.random.choice(np.arange(finalProb.shape[0]), p=finalProb))
        answers.append(l)
    return (np.bincount(answers[4500:]))

In [100]:
acc = []
for i in range(200):
    pred = predict(brain[i])
    acc.append(np.argmax(pred))
#     print(i, pred/np.sum(pred))

In [133]:
acc = np.array(acc)
accuracy = (np.sum(acc[:500]==0) + np.sum(acc[500:1000]==1) + np.sum(acc[1000:1500]==2) + np.sum(acc[1500:]==3))/2000
accuracy

0.8440714285714286

In [ ]:
############################################ END #########################################################

In [ ]:
#show_streamlines(cst)

In [ ]:
#show_multiple_streamlines(bundle,cst)

let's see its data type

In [ ]:
#bundle # whole bundle, made of several streamlines

In [ ]:
# ((brain)[0]).shape# number of streamlines in bundle

In [ ]:
# bundle[1] #one streamline in bundle
# (np.array(bundle)[0]).shape
# np.array(bundle)

In [ ]:
# (bundle[1].shape) #one point on one streamline

In [117]:
npbrain = np.array(brain)

In [ ]:
# npbrain[0].shape

In [ ]:
# len(bundle)

these are x, y and z co-ordinates of the point

In [97]:
# get labels

def getLabels(brain, bundle):
    labels = np.zeros(len(brain))
    npbrain = np.array(brain)
    for i in range(len(bundle)):
        for j in range(len(brain)):
            if((npbrain[j].shape[0]==bundle[i].shape[0])):
    #         print(i,j)
                if np.allclose(npbrain[j], bundle[i]):
                    labels[j] = 1
                    break
        if(i%100==0):
            print(i,end=" ")
    return labels